In [68]:
dateformat = "%Y-%m-%d"

import datetime

ts = spark.sql("select current_timestamp() as ctime").collect()[0]["ctime"]

todaydate = ts.strftime(dateformat)

StatementMeta(SparkPool, 5, 8, Finished, Available)

In [69]:
from pyspark.sql.functions import *
from notebookutils import mssparkutils

StatementMeta(SparkPool, 5, 9, Finished, Available)

In [70]:
sourceaccountname = "ottadls011"
sourcecontainer = "raw"
sourcelinkedService = "LS_ADLS_Transform"
sourceFile_location = "ingest/"+todaydate

StatementMeta(SparkPool, 5, 10, Finished, Available)

In [71]:
print(sourceFile_location)

StatementMeta(SparkPool, 5, 11, Finished, Available)

ingest/2023-10-25


In [72]:
spark.conf.set(f"spark.storage.synapse.linkedServiceName", "LS_ADLS_Transform")
spark.conf.set(f"fs.azure.account.oauth.provider.type", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")

path = f'abfss://{sourcecontainer}@{sourceaccountname}.dfs.core.windows.net/'

print(path)

StatementMeta(SparkPool, 5, 12, Finished, Available)

abfss://raw@ottadls011.dfs.core.windows.net/


In [73]:
file = f'abfss://{sourcecontainer}@{sourceaccountname}.dfs.core.windows.net/{sourceFile_location}'
print(file)

StatementMeta(SparkPool, 5, 13, Finished, Available)

abfss://raw@ottadls011.dfs.core.windows.net/ingest/2023-10-25


In [74]:
source_df = spark.read.format('csv').option("header","true").option("inferSchema","true").load(file)

StatementMeta(SparkPool, 5, 14, Finished, Available)

In [75]:
source_df.show()

StatementMeta(SparkPool, 5, 15, Finished, Available)

+--------------------+--------------------+------------------+-------+----------+--------+------+----------+
|               Title|               Genre|       ReleaseDate|Runtime|IMDB Score|Language| Views| AddedDate|
+--------------------+--------------------+------------------+-------+----------+--------+------+----------+
|      The Highwaymen|         Crime drama|    March 29, 2019|    131|       6.9| English| 54216|23-02-2023|
|The Lonely Island...|    Comedy / Musical|      May 23, 2019|     30|       6.9| English|238789|23-02-2023|
|The Meyerowitz St...|        Comedy-drama|  October 13, 2017|    112|       6.9| English| 48297|23-02-2023|
|Feminists: What W...|         Documentary|  October 12, 2018|     86|       7.0| English|494819|23-02-2023|
| Gaga: Five Foot Two|         Documentary|September 22, 2017|    100|       7.0| English|382459|23-02-2023|
|I'm No Longer Her...|Aftershow / Inter...|  November 3, 2020|     14|       7.0| English|711223|23-02-2023|
|       Kingdom of 

In [76]:
#Identifying duplicates

source_df.groupBy(source_df.columns).count().filter(col("count")>1).show()

StatementMeta(SparkPool, 5, 16, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+-----+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|count|
+-----+-----+-----------+-------+----------+--------+-----+---------+-----+
+-----+-----+-----------+-------+----------+--------+-----+---------+-----+



In [77]:
df_nodups = source_df

StatementMeta(SparkPool, 5, 17, Finished, Available)

In [78]:
df_nodups = df_nodups.drop_duplicates()

StatementMeta(SparkPool, 5, 18, Finished, Available)

In [79]:
#Identifying duplicates

df_nodups.groupBy(source_df.columns).count().filter(col("count")>1).show()

StatementMeta(SparkPool, 5, 19, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+-----+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|count|
+-----+-----+-----------+-------+----------+--------+-----+---------+-----+
+-----+-----+-----------+-------+----------+--------+-----+---------+-----+



In [80]:
# checking for NULL value Error

df_nodups.filter(df_nodups.Title.isNull()).show()

StatementMeta(SparkPool, 5, 20, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [81]:
df_nodups.select(\
    [count(\
        when(isnan(c)  | col(c).isNull(), c)\
          ).alias(c) for c in df_nodups.columns]
        ).show()

StatementMeta(SparkPool, 5, 21, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
|    0|    0|          0|      0|         0|       0|    0|        0|
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [82]:
# drop only if entire row is NULL

df_nodups = df_nodups.na.drop(how="all")

StatementMeta(SparkPool, 5, 22, Finished, Available)

In [83]:
# -- Replacing Null ValuesView

df_nonnulls = df_nodups.na.fill(value="Unknown")

StatementMeta(SparkPool, 5, 23, Finished, Available)

In [84]:
df_nonnulls.filter(col('Language').isNull()).show()

StatementMeta(SparkPool, 5, 24, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [85]:
df_nonnulls.filter(col('Genre').isNull()).show()

StatementMeta(SparkPool, 5, 25, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [86]:
df_nonnulls.filter(col('Language') == "Unknown").show()

StatementMeta(SparkPool, 5, 26, Finished, Available)

+-----+-----+-----------+-------+----------+--------+-----+---------+
|Title|Genre|ReleaseDate|Runtime|IMDB Score|Language|Views|AddedDate|
+-----+-----+-----------+-------+----------+--------+-----+---------+
+-----+-----+-----------+-------+----------+--------+-----+---------+



In [87]:

df_nonnulls = df_nonnulls.withColumn("IMDB Category",
                       when(col("IMDB Score").between(0.1, 2.9), "Very Low")
                       .when(col("IMDB Score").between(3, 4.9), "Low")
                       .when(col("IMDB Score").between(5, 7.9), "Medium")
                       .otherwise("High"))


StatementMeta(SparkPool, 5, 27, Finished, Available)

In [88]:
display(df_nonnulls)

StatementMeta(SparkPool, 5, 28, Finished, Available)

SynapseWidget(Synapse.DataFrame, 514d5d02-c91a-4306-ba1c-c50738e9ad09)

In [89]:
# New Coulumn runtime in hours

df = df_nonnulls.withColumn("RuntimeinHours",
                        round(df_nonnulls.Runtime/60,2))

StatementMeta(SparkPool, 5, 29, Finished, Available)

In [90]:
display(df)

StatementMeta(SparkPool, 5, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4bdafc11-9f7b-4511-b0fe-67a4e1a74078)

In [91]:
df = df.drop("Runtime")

StatementMeta(SparkPool, 5, 31, Finished, Available)

In [ ]:
df.printSchema()

StatementMeta(, , , Waiting, )

In [ ]:
## Changing String type to Date type

df = df.withColumn("ReleaseDate",to_date("ReleaseDate",'dd-MM-yyyy'))

StatementMeta(, , , Waiting, )

In [ ]:
## Changing String type to Date type

df = df.withColumn("AddedDate",to_date("ReleaseDate",'dd-MM-yyyy'))

StatementMeta(, , , Waiting, )

In [ ]:
df.printSchema()

StatementMeta(, , , Waiting, )

In [ ]:
df = df.withColumnRenamed("IMDB Score","IMDB_Score")

StatementMeta(, , , Waiting, )

In [ ]:
df = df.withColumnRenamed("IMDB Category","IMDB_Category")

StatementMeta(, , , Waiting, )

In [ ]:
display(df)

StatementMeta(, , , Waiting, )

In [ ]:
destaccountName = "ottadls011"
destcontainer = "refined"
destinationFolder = "data/"

StatementMeta(, , , Waiting, )

In [ ]:
## Writing Data to ADLS
destpath = f'abfss://{destcontainer}@{destaccountName}.dfs.core.windows.net/{destinationFolder}'


df.write\
    .mode("overwrite")\
    .format("parquet")\
    .option("header","true")\
    .option("inferschema","true")\
    .save(destpath)

StatementMeta(, , , Waiting, )